In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
path = '/content/drive/My Drive/dsn2021/'

In [ ]:
import pandas as pd
import numpy as np
import datetime

from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.model_selection import StratifiedKFold

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder

from sklearn.ensemble import RandomForestClassifier

from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.model_selection import StratifiedKFold

# **TRAIN PREPROCESSING**

In [ ]:
train = pd.read_csv(path + '/dataset/Train.csv')

In [ ]:
display(train.head())

In [ ]:
target = train['Response']

In [ ]:
target.value_counts()

In [ ]:
train.drop('Response', axis = 1, inplace = True)

In [ ]:
train.drop('ID', axis = 1, inplace = True)
train.shape

In [ ]:
train['Date_Customer'] = pd.to_datetime(train['Date_Customer'])

In [ ]:
train['Date_Customer'] = (datetime.datetime.now() - train['Date_Customer']).dt.days

In [ ]:
train['Year_of_Birth'] = pd.to_datetime(train['Year_of_Birth'], format = '%Y')

In [ ]:
train['Year_of_Birth'] = datetime.datetime.now().year - train['Year_of_Birth'].dt.year

In [ ]:
target.value_counts().plot(kind='bar')

In [ ]:
train['Disposable_Income'].fillna(np.median(train['Disposable_Income']), inplace = True)

In [ ]:
edu_dict = {'Basic' : 1,
            '2n Cycle' : 2,
            'Graduation' : 3,
            'Master' : 4,
            'PhD': 5
            }
train['Education_Level'] = train['Education_Level'].replace(edu_dict)

In [ ]:
encoder = LabelEncoder()

train['Marital_Status'] = encoder.fit_transform(train['Marital_Status'])

In [ ]:
train.dtypes

# **TEST PREPROCESSING**

In [ ]:
test = pd.read_csv(path + '/dataset/Test.csv')

In [ ]:
ID = test['ID']

In [ ]:
test['Date_Customer'] = pd.to_datetime(test['Date_Customer'])

In [ ]:
test['Date_Customer'] = (datetime.datetime.now() - test['Date_Customer']).dt.days

In [ ]:
test['Year_of_Birth'] = pd.to_datetime(test['Year_of_Birth'], format = '%Y')

In [ ]:
test['Year_of_Birth'] = datetime.datetime.now().year - test['Year_of_Birth'].dt.year

In [ ]:
test['Disposable_Income'].fillna(np.median(train['Disposable_Income']) , inplace = True)

In [ ]:
test['Education_Level'] = test['Education_Level'].replace(edu_dict)

In [ ]:
test['Marital_Status'] = encoder.transform(test['Marital_Status'])

In [ ]:
test.drop('ID', axis = 1, inplace = True)
test.shape

In [ ]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(train, target, test_size=0.2, random_state=42,)

In [ ]:
folds = 10
skf = StratifiedKFold(n_splits=folds, shuffle = True, random_state = 1001)

# **XGBOOST**

In [ ]:
from xgboost import XGBClassifier

In [ ]:
xgb = XGBClassifier(scale_pos_weight=5.561, objective = 'binary:logistic', booster = 'gbtree',
                    colsample_bytree =  0.3, gamma = 0.9, max_depth = 5, min_child_weight = 2, n_estimators = 2000, subsample = 0.9)

In [ ]:
xgb.fit(train, target,
        # eval_set = [(x_train, y_train),(x_test, y_test)],
        eval_metric = 'logloss',
        # early_stopping_rounds=10,
        )

# **CATBOOST**

In [ ]:
!pip install catboost

In [ ]:
from catboost import CatBoostClassifier

In [ ]:
cat = CatBoostClassifier(scale_pos_weight = 5.561, )

In [ ]:
cat_params = {"iterations" : [500, 600, 1000, 1500, 2000],
           "learning_rate" : [0.02, 0.03, 0.04],
           "depth" : [4, 5, 6, 8, 9, 10],
           "l2_leaf_reg" : np.logspace(-20, -19, 3),
           "leaf_estimation_iterations" : [10],
           }

# Searching through catboostclassifier
folds = 10
param_comb = 10
cat_search = RandomizedSearchCV(cat,
                                param_distributions = cat_params,
                                n_iter = param_comb,
                                scoring = "f1",
                                n_jobs = -1,
                                cv = skf.split(x_train,y_train),
                                verbose = 3,
                                random_state = 1001,
                                )

# Fitting the search to out data with nan
cat_search.fit(x_train, y_train)

In [ ]:
cat_search.best_params_

In [ ]:
cat = CatBoostClassifier(scale_pos_weight = 5.561, depth= 4, learning_rate = 0.02)

In [ ]:
cat.fit(train, target)

# **EVALUATION**

In [ ]:
from sklearn.metrics import f1_score
print(f1_score(y_test, cat_search.predict(x_test)))

In [ ]:
print(cat_search.score(x_train, y_train))
print(cat_search.score(x_test, y_test))

In [ ]:
pred = cat.predict(test)

In [ ]:
submit = pd.DataFrame({'ID' : ID,
              'Response' : pred })

In [ ]:
submit.to_csv(path + '/dataset/28th_submission.csv', index = False)